In [27]:
import vectice as vct

vec_project = vct.connect(config="token_i.json")

Welcome, 'Ines Cerdan'. You`re now successfully connected to the project '_FFBank & LuxAir Co-Branded Credit Card Initial offer' in Vectice.

To access a specific phase, use project.phase(Phase ID)
To get a list of phases you can access and their IDs, use project.list_phases()

For quick access to the list of phases in the Vectice web app, visit:
https://app.vectice.com/browse/project/PRJ-189


### Read datasets:
 - PTY_ID_MAIN - From our BigQuery Dev environment
 - HIST_TRANS - From our BigQuery Dev environment
 - LuxAir_Accts - S3
 - OFAC_SDN - S3

In [28]:
# Connect to BigQuery Dev using Service Account
from google.cloud import bigquery
from google.oauth2 import service_account
creds = service_account.Credentials.from_service_account_file("bq_dev_sa.json", scopes=["https://www.googleapis.com/auth/cloud-platform"])

bigquery_client = bigquery.Client(
    credentials= creds,
    project=creds.project_id
)

Query PTY_ID_MAIN and HIST_TRX from our Dev BigQUery env. Retrieving full tables, we will need to remove non US customers from the resultsets as per compliance.

In [29]:
# Query PTY_ID_MAIN table
qry_PTY_ID_MAIN = "SELECT * FROM `solutions-engineering-363108.CUST_PTY_INFO.PTY_ID_MAIN`"
#Run the query and write result to a pandas data frame
Query_Results = bigquery_client.query(qry_PTY_ID_MAIN)
df_PTY_ID_MAIN = Query_Results.to_dataframe()
#View top few rows of result
df_PTY_ID_MAIN.head()

,Customer_PTY_Id,Customer_Surname,Customer_Given_Name,Customer_Email,Customer_Gender,Customer_DOB,Customer_POB,Customer_Street_Address,Customer_State_Address,Customer_Zip_Address,Customer_Income,Customer_SSN,Customer_ITIN,Customer_Employer,Customer_Account_Open_Date,Customer_Initial_Deposit,Customer_Address_Cntry
0,65-3683456,Ney,Alf,aney36@wikipedia.org,Male,1999-02-14,Brea,6760 Corry Hill,CA,92822,229399.52,178-89-2425,576-23-1520,Muxo,1996-07-28,7736.97,US
1,06-0539217,Corcoran,Kev,kcorcoran1p@wp.com,Male,1987-02-05,Brea,2 Dryden Terrace,CA,92822,1109648.60,752-91-3542,646-53-1152,Kare,2005-12-12,60650.37,US
2,33-8558058,Doxsey,Julian,jdoxsey8l@mit.edu,Male,1964-03-01,Brea,30 Stephen Crossing,CA,92822,254149.73,636-48-4129,160-12-9075,Tambee,2004-03-26,14382.47,US
3,19-2047408,Thies,Grady,gthies2q@addthis.com,Male,2002-10-26,Erie,6414 Harper Junction,PA,16550,1079876.08,835-02-3608,295-66-1574,Yabox,2008-01-22,2823.16,US
4,08-0960611,O' Meara,Desi,domearaqd@bing.com,Male,1928-03-12,Erie,3 Melby Hill,PA,16550,1203616.91,567-95-1426,685-08-6154,Divape,1997-12-16,9901.73,US


In [30]:
# Query HIST_TRANS table
qry_HIST_TRX = "SELECT * FROM `solutions-engineering-363108.HIST_CUST_INFO.HIST_TRANS`"
#Run the query and write result to a pandas data frame
Query_Results = bigquery_client.query(qry_HIST_TRX)
df_HIST_TRX = Query_Results.to_dataframe()
#View top few rows of result
df_HIST_TRX.head()

,TransactionID,Customer_PTY_ID,AccountNumber,Email,Gender,Amount,Date,TransactionType
0,768,97-0539166,GT46 XFXO VIXP SUMO O1AE WXIJ K6G7,gismead2x@google.ru,Male,515287.07,2017-03-14,Wire
1,768,91-4224986,TN58 1132 2103 0923 2635 4596,helcoux4b@ucla.edu,Male,113210.26,2017-08-14,Wire
2,768,55-6039702,RS97 6933 0668 4971 9957 29,adimitruea@rambler.ru,Female,769819.63,2017-12-17,Wire
3,768,04-6258749,AD12 7772 3942 OKBX BHWK MPY2,mmunningsh3@hugedomains.com,Female,48354.43,2018-02-01,Wire
4,768,82-4918676,PL37 5346 9955 0710 9093 8989 3441,ldengefv@digg.com,Male,237134.26,2018-11-03,Wire


Reading the two external files from our S3 bucket.

In [31]:
# Read the external files from S3
# Create connection
from boto3 import client
from botocore import UNSIGNED
from botocore.client import Config
import s3fs

s3_client = client('s3', config=Config(signature_version=UNSIGNED), region_name='us-west-1')


In [32]:
import pandas as pd

# Read the external files in dataframes
s3 = s3fs.S3FileSystem(anon=True)

with s3.open("vectice-examples/Samples Data/LuxAir_Accts.csv", mode="rb") as f:
    df_LuxAir_Accts = pd.read_csv(f)

with s3.open("vectice-examples/Samples Data/OFAC_SDN.csv", mode="rb") as f:
    df_OFAC_SDN = pd.read_csv(f)

Drop features that are not needed, create target, split LuxAIr_Accts.CUST_NAME into CUST_F_NAME and CUST_L_NAME.

In [33]:
# Prep LuxAir dataset
# Remove NAs
df_LuxAir_Accts = df_LuxAir_Accts.dropna()
# Split columns
df_LuxAir_Accts[['CUST_F_NAME', 'CUST_L_NAME']] = df_LuxAir_Accts['CUST_NAME'].str.split(pat=' ', n=1,expand=True)
# Drop features
df_LuxAir_Accts = df_LuxAir_Accts.drop(columns=['CUST_NAME','CUST_PREFS'])
df_LuxAir_Accts.shape[0]



2409

In [34]:
# Prep PTY_ID_MAIN dataset
# Remove NAs
df_PTY_ID_MAIN = df_PTY_ID_MAIN.dropna()
# Only keep US accts
df_PTY_ID_MAIN = df_PTY_ID_MAIN.loc[df_PTY_ID_MAIN["Customer_Address_Cntry"] == 'US' ]
df_PTY_ID_MAIN.shape[0]

4604

In [35]:
# Join the datasets to create a clean dataset ready for modeling
df_JOINED_DS = pd.merge(df_PTY_ID_MAIN, df_LuxAir_Accts, how='inner', left_on='Customer_POB', right_on = 'CUST_CITY')

df_JOINED_DS.head()

,Customer_PTY_Id,Customer_Surname,Customer_Given_Name,Customer_Email,Customer_Gender,Customer_DOB,Customer_POB,Customer_Street_Address,Customer_State_Address,Customer_Zip_Address,...,CUST_ZIP,CUST_COUNTRY,CUST_CTRY_CD,CUST_PHONE,YTD_STATUS,YTD_MILES,YTD_SEGMENTS,ORI_AIRPORT,CUST_F_NAME,CUST_L_NAME
0,94-4198653,Dee,Fabiano,fdeez@istockphoto.com,Male,1997-12-07,Gary,5293 Manufacturers Court,IN,46406,...,46406,United States,US,219-809-4413,Diamond,8883383,81,US-GA,Anni,Learmonth
1,47-2882119,Skivington,Giraud,gskivington1k@domainmarket.com,Male,2006-11-21,Gary,47861 Merrick Street,IN,46406,...,46406,United States,US,219-809-4413,Diamond,8883383,81,US-GA,Anni,Learmonth
2,31-6155048,Critzen,Sarge,scritzen2a@newyorker.com,Male,1924-05-29,Gary,5 Ridgeview Crossing,IN,46406,...,46406,United States,US,219-809-4413,Diamond,8883383,81,US-GA,Anni,Learmonth
3,25-8144215,Gallie,Debi,dgallie3e@unesco.org,Female,1979-07-15,Gary,51 Debs Hill,IN,46406,...,46406,United States,US,219-809-4413,Diamond,8883383,81,US-GA,Anni,Learmonth
4,10-8050238,Bramo,Nicoli,nbramoc1@google.com.hk,Female,1998-09-11,Gary,2874 Southridge Plaza,IN,46406,...,46406,United States,US,219-809-4413,Diamond,8883383,81,US-GA,Anni,Learmonth


In [36]:
import numpy as np
df_JOINED_DS["accepted_offer"] = np.where((df_JOINED_DS['YTD_SEGMENTS'] > 45), 0, 1)
df_JOINED_DS.head()

,Customer_PTY_Id,Customer_Surname,Customer_Given_Name,Customer_Email,Customer_Gender,Customer_DOB,Customer_POB,Customer_Street_Address,Customer_State_Address,Customer_Zip_Address,...,CUST_COUNTRY,CUST_CTRY_CD,CUST_PHONE,YTD_STATUS,YTD_MILES,YTD_SEGMENTS,ORI_AIRPORT,CUST_F_NAME,CUST_L_NAME,accepted_offer
0,94-4198653,Dee,Fabiano,fdeez@istockphoto.com,Male,1997-12-07,Gary,5293 Manufacturers Court,IN,46406,...,United States,US,219-809-4413,Diamond,8883383,81,US-GA,Anni,Learmonth,0
1,47-2882119,Skivington,Giraud,gskivington1k@domainmarket.com,Male,2006-11-21,Gary,47861 Merrick Street,IN,46406,...,United States,US,219-809-4413,Diamond,8883383,81,US-GA,Anni,Learmonth,0
2,31-6155048,Critzen,Sarge,scritzen2a@newyorker.com,Male,1924-05-29,Gary,5 Ridgeview Crossing,IN,46406,...,United States,US,219-809-4413,Diamond,8883383,81,US-GA,Anni,Learmonth,0
3,25-8144215,Gallie,Debi,dgallie3e@unesco.org,Female,1979-07-15,Gary,51 Debs Hill,IN,46406,...,United States,US,219-809-4413,Diamond,8883383,81,US-GA,Anni,Learmonth,0
4,10-8050238,Bramo,Nicoli,nbramoc1@google.com.hk,Female,1998-09-11,Gary,2874 Southridge Plaza,IN,46406,...,United States,US,219-809-4413,Diamond,8883383,81,US-GA,Anni,Learmonth,0


Save the clean dataset in BQ dev

In [37]:
table_id = 'FFBank_CoBranded_project.modelingdata'

# Since string columns use the "object" dtype, pass in a (partial) schema
# to ensure the correct BigQuery data type.
job_config = bigquery.LoadJobConfig(schema=[
    bigquery.SchemaField("Customer_PTY_Id", "STRING"),
])

job = bigquery_client.load_table_from_dataframe(
    df_JOINED_DS, table_id, job_config=job_config
)

# Vectice Documentation Pipeline

In [38]:
from vectice import Dataset, S3Resource
from vectice.models.resource import BigQueryResource

iteration = vec_project.phase("Data Analysis").create_or_get_current_iteration()

Phase 'Data Analysis' successfully retrieved."

For quick access to the Phase in the Vectice web app, visit:
https://app.vectice.com/browse/phase/PHA-1076
Iteration number 6 successfully retrieved.

For quick access to the Iteration in the Vectice web app, visit:
https://app.vectice.com/browse/iteration/ITR-3088


In [39]:
# Log insights in Vectice
msg = "The data was prepared according to the Bank's data pipeline for:\n\tcompletness\n\tnormalization\n\tsecurity\n\tbias free\n\tanonymized\nAs required by Compliance, we removed non US addresses.\nRows with NA values from both the LuxAir_Accts and the PTY_ID_MAIN dataframes were dropepd.\n"\
"We split the LuxAir CUST_NAME into CUST_F_NAME and CUST_L_NAME, dropped CUST_NAME and CUST_PREFS.\n"

iteration.step_construct_data = msg

# Add the clean dataset to Vectice

vct_JOINED_DS = BigQueryResource(bq_client=bigquery_client, paths="solutions-engineering-363108." + table_id, dataframes = df_JOINED_DS)
iteration.step_integrate_data += Dataset.clean(name="Clean dataset", resource=vct_JOINED_DS,  properties={"BQ Table Name":table_id})

iteration.step_integrate_data += "We merged the datasets into a clean ready to model datasets using an inner join on PTY_ID_MAIN on CUST_NAME, CUST_ADDRESS and saved the dataset on our BigQuery environment as \"modelingdata\"."

iteration.step_format_data += "Data formatted as per Bank's standards. Automated Data pipeline process needs to be implemented"

iteration.step_certify_compliance += f"For compliance validation:\nList of unique customer address country: {df_JOINED_DS.Customer_Address_Cntry.unique()}"\
    f"\nThe LuxAir dataset has been validated against the OFAC list for proper due diligence and KYC. Removed one account(s)."

iteration.complete()

<>:16: DeprecationWarning: invalid escape sequence \F
<>:16: DeprecationWarning: invalid escape sequence \F
/var/folders/3_/l7f8whx5221gn40trqt3h6l80000gn/T/ipykernel_20441/282792939.py:16: DeprecationWarning: invalid escape sequence \F
  iteration.step_certify_compliance += f"\For compliance validation:\nList of unique customer address country: {df_JOINED_DS.Customer_Address_Cntry.unique()}"\
Added Comment to Step: Construct Data

Link to Step: https://app.vectice.com/browse/iteration/ITR-3088

Existing Dataset: 'Clean dataset' and Version: 'Version 6' already linked to Step: Integrate Data
Attachments: None
Link to Step: https://app.vectice.com/browse/iteration/ITR-3088

Added Comment to Step: Integrate Data

Link to Step: https://app.vectice.com/browse/iteration/ITR-3088

Added Comment to Step: Format Data

Link to Step: https://app.vectice.com/browse/iteration/ITR-3088

Added Comment to Step: Certify Compliance

Link to Step: https://app.vectice.com/browse/iteration/ITR-3088

Itera